In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

In [2]:
combined_df = pd.read_csv('./data/regression.csv')

# Convert 'Date' column to pandas datetime and set it as the index
combined_df['date'] = pd.to_datetime(combined_df['date'])
combined_df.set_index('date', inplace=True)

combined_df.head(5)

,pool_count,currency_pairs,daily_pools_created,volatility_BTC,volatility_ETH,distinct_crypto_count,direct_pairs,usdt_pairs,dai_pairs,usdc_pairs,wbtc_pairs,weth_pairs
date,,,,,,,,,,,,
2020-05-18,9,"REN/USDC, CHAI/WETH, PAX/USDC, BNT/DAI, DAI/WE...",0.0,0.663055,0.651359,9,9,0,3,4,0,5
2020-05-19,9,"REN/USDC, CHAI/WETH, PAX/USDC, BNT/DAI, DAI/WE...",0.0,0.657939,0.637882,9,9,0,3,4,0,5
2020-05-20,192,"WBTC/USDC, XOR/WETH, DGX/WETH, renZEC/DAI, WET...",183.0,0.649254,0.617111,143,171,5,26,21,6,121
2020-05-21,244,"WBTC/USDC, XOR/WETH, DGX/WETH, FUN/WETH, renZE...",52.0,0.670603,0.642146,183,222,6,36,27,6,155
2020-05-22,271,"WBTC/USDC, XOR/WETH, LMY/WETH, DGX/WETH, FUN/W...",27.0,0.664072,0.623359,210,247,6,39,27,6,177


In [3]:
# Get the end date from the index
end_date = combined_df.index.max()
print("End date:", end_date)

End date: 2024-12-20 00:00:00


In [4]:
# Define the predictors and the target variable
predictors = [
    'volatility_BTC', 'volatility_ETH',
    'usdt_pairs', 'dai_pairs', 'weth_pairs'
]
target = 'daily_pools_created'

For each regressor, I'll need to obtain an estimate of values that'll occur in Feb 2025 to estimate the number of liquidity pools in Feb 2025.  

In [5]:
# Generate future dates up to February 1, 2025
future_dates = pd.date_range(start=combined_df.index[-1] + pd.Timedelta(days=1), end='2025-02-01')
future_df = pd.DataFrame(index=future_dates)

# Combine the existing and future DataFrames
extended_df = pd.concat([combined_df, future_df])

extended_df

,pool_count,currency_pairs,daily_pools_created,volatility_BTC,volatility_ETH,distinct_crypto_count,direct_pairs,usdt_pairs,dai_pairs,usdc_pairs,wbtc_pairs,weth_pairs
2020-05-18,9.0,"REN/USDC, CHAI/WETH, PAX/USDC, BNT/DAI, DAI/WE...",0.0,0.663055,0.651359,9.0,9.0,0.0,3.0,4.0,0.0,5.0
2020-05-19,9.0,"REN/USDC, CHAI/WETH, PAX/USDC, BNT/DAI, DAI/WE...",0.0,0.657939,0.637882,9.0,9.0,0.0,3.0,4.0,0.0,5.0
2020-05-20,192.0,"WBTC/USDC, XOR/WETH, DGX/WETH, renZEC/DAI, WET...",183.0,0.649254,0.617111,143.0,171.0,5.0,26.0,21.0,6.0,121.0
2020-05-21,244.0,"WBTC/USDC, XOR/WETH, DGX/WETH, FUN/WETH, renZE...",52.0,0.670603,0.642146,183.0,222.0,6.0,36.0,27.0,6.0,155.0
2020-05-22,271.0,"WBTC/USDC, XOR/WETH, LMY/WETH, DGX/WETH, FUN/W...",27.0,0.664072,0.623359,210.0,247.0,6.0,39.0,27.0,6.0,177.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For each predictors, I extrapolate each time-series straight forwardly. Here I use a 60-day moving average for extrapolation.

In [6]:
# Calculate 60-day moving average for predictors
for predictor in predictors:
    extended_df[predictor] = extended_df[predictor].rolling(60, min_periods=1).mean()

extended_df

,pool_count,currency_pairs,daily_pools_created,volatility_BTC,volatility_ETH,distinct_crypto_count,direct_pairs,usdt_pairs,dai_pairs,usdc_pairs,wbtc_pairs,weth_pairs
2020-05-18,9.0,"REN/USDC, CHAI/WETH, PAX/USDC, BNT/DAI, DAI/WE...",0.0,0.663055,0.651359,9.0,9.0,0.000000,3.000000,4.0,0.0,5.000000
2020-05-19,9.0,"REN/USDC, CHAI/WETH, PAX/USDC, BNT/DAI, DAI/WE...",0.0,0.660497,0.644620,9.0,9.0,0.000000,3.000000,4.0,0.0,5.000000
2020-05-20,192.0,"WBTC/USDC, XOR/WETH, DGX/WETH, renZEC/DAI, WET...",183.0,0.656749,0.635450,143.0,171.0,1.666667,10.666667,21.0,6.0,43.666667
2020-05-21,244.0,"WBTC/USDC, XOR/WETH, DGX/WETH, FUN/WETH, renZE...",52.0,0.660213,0.637124,183.0,222.0,2.750000,17.000000,27.0,6.0,71.500000
2020-05-22,271.0,"WBTC/USDC, XOR/WETH, LMY/WETH, DGX/WETH, FUN/W...",27.0,0.660985,0.634371,210.0,247.0,3.400000,21.400000,27.0,6.0,92.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-28,NaN,NaN,NaN,0.435032,0.622430,NaN,NaN,8.000000,4.000000,NaN,NaN,947.714286
2025-01-29,NaN,NaN,NaN,0.431292,0.620481,NaN,NaN,8.000000,4.000000,NaN,NaN,947.700000
2025-01-30,NaN,NaN,NaN,0.427386,0.618367,NaN,NaN,8.000000,4.000000,NaN,NaN,947.684211
2025-01-31,NaN,NaN,NaN,0.422824,0.615821,NaN,NaN,8.000000,4.000000,NaN,NaN,947.666667


Next, use Model 9 to predict the daily pools created February 1, 2025.

In [7]:
# Split the data into in-sample and out-of-sample (using a specific date)
split_date = '2023-01-01'

# Split the data
train_df = combined_df.loc[:split_date]
future_df = extended_df.loc['2024-12-21':'2025-02-01']

# Select predictors and target
X_train = train_df[predictors]
y_train = train_df[target]

# Add a constant term to the predictors
X_train_const = sm.add_constant(X_train)

# Fit the model using statsmodels
model = sm.OLS(y_train, X_train_const).fit()

# Print the summary of the model
print(model.summary())

                             OLS Regression Results                            
Dep. Variable:     daily_pools_created   R-squared:                       0.432
Model:                             OLS   Adj. R-squared:                  0.429
Method:                  Least Squares   F-statistic:                     144.9
Date:                 Mon, 23 Dec 2024   Prob (F-statistic):          2.19e-114
Time:                         15:27:27   Log-Likelihood:                -5566.7
No. Observations:                  959   AIC:                         1.115e+04
Df Residuals:                      953   BIC:                         1.117e+04
Df Model:                            5                                         
Covariance Type:             nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            266.2904     35.0

In [8]:
# Prepare future predictors
X_future = future_df[predictors]
X_future_const = sm.add_constant(X_future, has_constant='add')
# print(X_future_const)
# print("Columns in X_future_const:", X_future_const.columns)

# Predict daily pools created for future dates
future_predictions = model.predict(X_future_const)

future_predictions

2024-12-21    284.900407
2024-12-22    284.833649
2024-12-23    284.793118
2024-12-24    284.740963
2024-12-25    284.593749
2024-12-26    284.479071
2024-12-27    284.408417
2024-12-28    284.438446
2024-12-29    284.331056
2024-12-30    284.153164
2024-12-31    284.070619
2025-01-01    283.987886
2025-01-02    283.907351
2025-01-03    283.819538
2025-01-04    283.766830
2025-01-05    283.704347
2025-01-06    283.463953
2025-01-07    283.146569
2025-01-08    282.686430
2025-01-09    282.302255
2025-01-10    282.731854
2025-01-11    283.082766
2025-01-12    283.445537
2025-01-13    283.929707
2025-01-14    284.558616
2025-01-15    285.231935
2025-01-16    285.949053
2025-01-17    286.620535
2025-01-18    287.323749
2025-01-19    287.988203
2025-01-20    288.369241
2025-01-21    288.823003
2025-01-22    289.384338
2025-01-23    289.960648
2025-01-24    291.162195
2025-01-25    292.432473
2025-01-26    293.349883
2025-01-27    294.176543
2025-01-28    295.051081
2025-01-29    296.151143


Now, I make the final prediction for how many uniswap v2 pools will exist on 1st February 2025.

In [9]:
latest_total_pools = combined_df.loc['2024-12-20', 'pool_count']
print(f"total pools on 2024-12-20: {latest_total_pools}")

total pools on 2024-12-20: 393887


In [10]:
# Sum predicted daily pools from 2024-12-21 to 2025-02-01
cumulative_future_pools = future_predictions.loc['2024-12-21':'2025-02-01'].sum()

# Calculate the total number of pools on 2025-02-01
predicted_total_pools = latest_total_pools + cumulative_future_pools

# Round the predicted total pools to the nearest integer
predicted_total_pools_rounded = round(predicted_total_pools)

print(f"Predicted total pools on 2025-02-01: {predicted_total_pools_rounded}")

Predicted total pools on 2025-02-01: 406233
